In [ ]:
import librosa  # For loading audio and LPC calculation
import python_speech_features as psf  # For MFCC and PLP calculation
import matplotlib.pyplot as plt  # For plotting
import numpy as np  # For numerical operations
from scipy.signal import (
    lfilter,
)  # Although imported, not directly used in the provided LPC calculation method


def extract_features(audio_file):
    try:
        # Load audio with librosa for consistent processing
        # Resample to 16000 Hz and limit duration to 3 seconds
        y, sr = librosa.load(audio_file, sr=16000, duration=3)
        print(f"Audio loaded: {audio_file} with sample rate {sr} Hz")

        # --- MFCC Features ---
        # Calculate Mel-Frequency Cepstral Coefficients
        # numcep: number of cepstra desired
        # nfilt: number of filters in the filterbank
        # nfft: FFT size
        mfcc_feat = psf.mfcc(y, sr, numcep=13, nfilt=26, nfft=512)
        print(f"MFCC Features extracted with shape: {mfcc_feat.shape}")

        # --- PLP Features ---
        # Calculate Perceptual Linear Prediction features
        # numcep: number of cepstra desired
        # nfilt: number of filters in the filterbank
        # nfft: FFT size
        plp_feat = psf.plp(y, sr, numcep=13, nfilt=26, nfft=512)
        print(f"PLP Features extracted with shape: {plp_feat.shape}")

        # --- LPC Features ---
        # Calculate Linear Predictive Coding coefficients per frame
        n_coeff = 12  # Number of LPC coefficients (order of the LPC model)
        lpc_coeffs = []

        # Frame the audio signal
        frame_length = 0.025 * sr  # 25 ms frame length
        frame_stride = 0.01 * sr  # 10 ms frame stride

        # Iterate through frames to calculate LPC for each
        # Adding a small amount of noise (0.001) to avoid potential issues with silence frames
        for frame in psf.sigproc.framesig(
            y, frame_length, frame_stride, winfunc=lambda x: np.ones((x,))
        ):
            # librosa.lpc computes the AR coefficients using the Yule-Walker equations
            # Adding a small epsilon * random noise to prevent division by zero or instability
            a = librosa.lpc(frame + 0.001 * np.random.randn(len(frame)), order=n_coeff)
            lpc_coeffs.append(a)

        lpc_feat = np.array(lpc_coeffs)
        print(f"LPC Features extracted with shape: {lpc_feat.shape}")

        return mfcc_feat, plp_feat, lpc_feat

    except FileNotFoundError:
        print(f"Error: Audio file not found at {audio_file}")
        return None, None, None
    except Exception as e:
        print(f"An error occurred during feature extraction: {e}")
        return None, None, None


def plot_features(mfcc, plp, lpc):
    if mfcc is None or plp is None or lpc is None:
        print("Cannot plot features: Feature extraction failed.")
        return

    plt.figure(figsize=(15, 10))

    # --- MFCC Plot ---
    plt.subplot(3, 1, 1)
    # Use imshow to visualize the feature matrix as an image
    # Transpose MFCC to have time on x-axis and coefficients on y-axis
    # aspect='auto' adjusts aspect ratio
    # origin='lower' places the origin at the bottom left
    # extent sets the x and y axis limits based on time and number of coefficients
    plt.imshow(
        mfcc.T,
        aspect="auto",
        origin="lower",
        extent=[0, mfcc.shape[0] * 0.01, 0, mfcc.shape[1]],
    )  # Assuming 10ms frame stride
    plt.title("MFCC Features")
    plt.ylabel("Cepstral Coefficient Index")
    plt.colorbar(label="Magnitude")

    # --- PLP Plot ---
    plt.subplot(3, 1, 2)
    # Similar to MFCC, visualize PLP features
    plt.imshow(
        plp.T,
        aspect="auto",
        origin="lower",
        extent=[0, plp.shape[0] * 0.01, 0, plp.shape[1]],
    )  # Assuming 10ms frame stride
    plt.title("PLP Features")
    plt.ylabel("Cepstral Coefficient Index")
    plt.colorbar(label="Magnitude")

    # --- LPC Plot ---
    plt.subplot(3, 1, 3)
    # Plot LPC coefficients over time (frames)
    # Skip the first coefficient (a[0]), which is typically 1
    plt.plot(lpc[:, 1:])
    plt.title("LPC Coefficients (a_1 to a_n)")
    plt.xlabel(
        "Frame Index"
    )  # Or time if calculated: Frame Index * frame_stride_in_seconds
    plt.ylabel("Coefficient Value")
    plt.grid(True)

    plt.tight_layout()  # Adjust layout to prevent overlapping titles/labels
    plt.savefig("feature_comparison.png")  # Save the plot to a file
    plt.show()  # Display the plot window


# --- Main Execution ---
if __name__ == "__main__":
    # Define the path to your audio file
    audio_file = (
        "speeches/sample.wav"  # Make sure you have this file in the same directory
    )

    # Extract features from the audio file
    mfcc, plp, lpc = extract_features(audio_file)

    # Print the shapes of the extracted features
    if mfcc is not None:
        print("MFCC Shape:", mfcc.shape)
    if plp is not None:
        print("PLP Shape:", plp.shape)
    if lpc is not None:
        print("LPC Shape:", lpc.shape)

    # Plot the extracted features
    plot_features(mfcc, plp, lpc)

    print("\nScript finished.")

Error: Audio file not found at speech_sample.wav
Cannot plot features: Feature extraction failed.

Script finished.


/tmp/ipykernel_35998/4250487006.py:14: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=16000, duration=3)
/home/parvez/reps/codes/disp_labs/.venv/lib/python3.13/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
